**Best Location for a New Mexican Restaurant**

*Import libraries, set-up notebook*

In [1]:
import numpy as np                                                       # library to handle data in a vectorized manner
from numpy.random import randint 

import pandas as pd                                                      # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json                                                              # library to handle JSON files
from pandas.io.json import json_normalize                                # tranform JSON file into a pandas dataframe
import requests                                                          # library to handle requests

from geopy.geocoders import Nominatim                                    # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="foursquare_agent")

import random                                                            # library for random number generation

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"                          # display requested dataframes in pandas format

print('Libraries imported.\n')

Libraries imported.



*Foursquare credentials*

In [2]:
CLIENT_ID = 'ZQKKMGNITTOGCMFEPVIQH5W1MZRDABZEX52IROB0KLFW1LVI'
CLIENT_SECRET = '2KOHRJGACJXUAK3BIRM1P0CORACXOJ15GPUJZQNHBNBH3MMP'
#CLIENT_ID = 'MJ1EJQRTUUE3ZGU4NE2DV2P2HZWO0U5E3QU1EFLOT51GQQC3'           # your Foursquare ID
#CLIENT_SECRET = 'UFY2KX53GD0PRWMZQ0T4BU2ANOHX24RZM2HGXNHAWODLRGL1'       # your Foursquare Secret
print('Your credentials are saved.\n')

Your credentials are saved.



*Prep variable assignment and cleanup for Foursquare endpoint*

In [3]:
VERSION = '20201015'

ri = 'response.venue.id'
rn = 'response.venue.name'
rr = "response.venue.rating"
rt = 'response.venue.price.tier'
rl = 'response.venue.likes.count'
rc = 'response.venue.location.city'
mr = 'mexRate'

# prep for next loop and fetch_ratings function 
mexReviews = pd.DataFrame(columns=['id','name','city','rating','tier','likes'])
entries = []

capMex = pd.read_csv(r'C:\Users\Jim_2\Desktop\python_projects\capMex.csv')

print('Your fixed variables are saved.\n')

Your fixed variables are saved.



In [4]:
#Loop through each row to get ratings, price tier and likes
for id, row in capMex.iterrows():
    venueID = row['id']
        
    # it returns a dataframe that provides the venue id, the rating factor, price tier and likes count
    url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venueID, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    global entry, entries, results2, mexRate, mexRate2                    # set global variables within function
    
    results2 = requests.get(url2).json()                                  # make the GET request
    mr = mexRate = pd.json_normalize(results2)                            # converts to dataframe
    
    # if statements to verify all fields being imported
    if ri not in mr: mr.insert(3,ri,'No ID')
    if rn not in mr: mr.insert(4,rn,'No name')
    if rc not in mr: mr.insert(5,rc,'No city')
    if rr not in mr: mr.insert(6,rr,"No rating")
    if rt not in mr: mr.insert(7,rt,"No tier")
    if rl not in mr: mr.insert(8,rl,"No likes")
    
    # select subset of mexRate and assign to a new dataframe: mexRate2 
    mexRate2 = mexRate[['response.venue.id','response.venue.name','response.venue.location.city','response.venue.rating','response.venue.price.tier','response.venue.likes.count']].copy()                                                        # creates new dataframe with required info
    mexRate2.columns = ['id','name','city','rating','tier','likes']      # rename columns
    
    entry = mexRate2.loc[0]                                              # save current venueID data to variable    
    entries.append(entry)                                                # append current venue to venues list 
        
mexTrans= pd.concat(entries)
mexReviews = pd.DataFrame (entries,columns=['id','name','city','rating','tier','likes'])
mexReviews.to_csv(r'C:\Users\Jim_2\Desktop\python_projects\mexReviews.csv', index=False, header=True) # saving file to disk
mexReviews.head()

,id,name,city,rating,tier,likes
0,4a46bd39f964a5205fa91fe3,Mexican Town Restaurant,Detroit,7.2,1,40
0,4ad36269f964a52012e420e3,Mexican Village Restaurant,Detroit,8.1,2,116
0,4b50da97f964a520663527e3,Qdoba Mexican Grill,Warren,7.4,1,34
0,4c615cb7de6920a1a5169864,La Salsa Mexican Taqueria,Detroit,8.4,2,23
0,4b104dcaf964a520656d23e3,On The Border Mexican Grill & Cantina,Allen Park,7.2,2,88
